# Install libraries

In [ ]:
pip install shap

In [ ]:
pip install lime

In [ ]:
pip install pycaret

# Import Packages

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pycaret
from pycaret.regression import *

import lime
import lime.lime_tabular

import shap

# Load dataset

In [ ]:
file_path = '/mnt/BlackBox_forecast_simulation.csv'
data = pd.read_csv(file_path)
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)
data = data.rename(columns={'BlackBox': 'blackbox_pred'})
data.head()

In [8]:
# Define datapoints of fourth fold for plotting
start_date = '1971-01-01'
end_date = '1971-12-01'
target = data[start_date:end_date]
target = target['sales']

In [26]:
# Drop the sales revenue time series
df = data.drop(columns=['sales'])

# Feature Engineering

In [ ]:
# Define function to create time index features
def create_ts_features(df):
    df = df.copy()
    df['month'] = df.index.month
    df['year'] = df.index.year
    return df

data_tsf = create_ts_features(df)
data_tsf.head()

In [28]:
# Create lag features
for i in range(1, 13):
  data_tsf['rev_lag' + str(i)] = data_tsf['blackbox_pred'].shift(i)

# Create rolling window features
for j in range(2, 13):
  data_tsf['r_avg_rev' + str(i)] = data_tsf['blackbox_pred'].rolling(j).mean().shift(1)
  data_tsf['r_sum_rev' + str(i)] = data_tsf['blackbox_pred'].rolling(j).sum().shift(1)
  data_tsf['r_min_rev' + str(i)] = data_tsf['blackbox_pred'].rolling(j).min().shift(1)
  data_tsf['r_max_rev' + str(i)] = data_tsf['blackbox_pred'].rolling(j).max().shift(1)

# Create expanding window features
data_tsf['exp_avg_rev'] = data_tsf['blackbox_pred'].expanding().mean()
data_tsf['exp_sum_rev'] = data_tsf['blackbox_pred'].expanding().sum()
data_tsf['exp_min_rev'] = data_tsf['blackbox_pred'].expanding().min()
data_tsf['exp_max_rev'] = data_tsf['blackbox_pred'].expanding().max()

In [ ]:
# Show dataframe with features
data_tsf.head()

# Model Training

In [30]:
# Create train and test folds
train = data_tsf.loc[data_tsf.index <= '1970-12-01']
test = data_tsf.loc[data_tsf.index > '1970-12-01']

In [ ]:
# Define the setup for pycaret including the normalize method, imputation type & and feature selection
s = setup(data = train,
          test_data = test,
          target = 'blackbox_pred',
          fold_strategy='timeseries',
          data_split_shuffle=False,
          fold_shuffle=False,
          fold=3,
          normalize=True,
          normalize_method='minmax',
          imputation_type='simple',
          numeric_imputation='mean',
          feature_selection=True,
          feature_selection_method='univariate',
          n_features_to_select=5,
          session_id = 123,
          )

In [32]:
# Create dataframes for train / test partitions
X_train = get_config('X_train')
X_train_transformed = get_config('X_train_transformed')
X_test = get_config('X_test')
X_test_transformed = get_config('X_test_transformed')

#Compare Models

In [ ]:
# Compare all models of the pycaret regression library
best = compare_models(
    sort = 'MAPE',
    cross_validation=True,
    n_select=5,
)

# Create Models

In [ ]:
# Create XGBoost
model1 = create_model("xgboost")

In [ ]:
# Tune XGBoost
model1 = tune_model(model1, n_iter = 50, optimize = 'MAPE', choose_better = True)

In [ ]:
# Plot feature importance
plot_model(model1, plot = 'feature')

In [ ]:
# Create Light Gradient Boosting Machine
model2 = create_model("lightgbm")

In [ ]:
# Tune LightGBM
model2 = tune_model(model2, n_iter = 50, optimize = 'MAPE', choose_better = True)

In [ ]:
# Plot feature importance
plot_model(model2, plot = 'feature')

## Create predictions

In [ ]:
# XGBoost: Create predictions for the test fold and plot the results
y_pred_1 = predict_model(model1)

plt.figure(figsize=(10, 6))

plt.plot(y_pred_1.index, target.values, label='sales revenue', marker='o', linestyle='-', color='#0072EF')
plt.plot(y_pred_1.index, y_pred_1['blackbox_pred'], label='blackbox prediction', marker='o', linestyle='-', color='#929EAB')
plt.plot(y_pred_1.index, y_pred_1['prediction_label'], label='surrogate prediction', marker='s', linestyle='-', color='#0F2DB3')

plt.title('Performance Fourth Fold XGBoost')
plt.xlabel('date')
plt.ylabel('value')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
# LightGBM: Create predictions for the test fold and plot the results
y_pred_2 = predict_model(model2)

plt.figure(figsize=(10, 6))

plt.plot(y_pred_2.index, target.values, label='sales revenue', marker='o', linestyle='-', color='#0072EF')
plt.plot(y_pred_2.index, y_pred_2['blackbox_pred'], label='blackbox prediction', marker='o', linestyle='-', color='#929EAB')
plt.plot(y_pred_2.index, y_pred_2['prediction_label'], label='surrogate prediction', marker='s', linestyle='-', color='#0F2DB3')


plt.title('Performance Fourth Fold lightgbm')
plt.xlabel('date')
plt.ylabel('value')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
# Plot sales time series / black box time series / predictions of XGBoost / predictions of LightGBM
plt.figure(figsize=(25, 6))

plt.plot(data.index, data['sales'], label='sales data', marker='o', linestyle='-', color='#929EAB')
plt.plot(data.index, data['blackbox_pred'], label='blackbox_predition', marker='o', linestyle='-', color='#0072EF')
plt.plot(y_pred_1.index, y_pred_1['prediction_label'], label='prediction_xgb', marker='s', linestyle='-', color='#0F2DB3')
plt.plot(y_pred_2.index, y_pred_2['prediction_label'], label='prediction_lightgbm', marker='s', linestyle='-', color='#4C6BB1')


for i in range(0, len(data), 12):
    plt.axvline(x=data.index[i], color='gray', linestyle='--', linewidth=2)

plt.title('Performance Evaluation ZEISS sales dataset')
plt.xlabel('date')
plt.ylabel('value')
plt.legend()
plt.grid(True)

plt.show()

# Explainable AI

## SHAP

In [ ]:
# Calculate SHAP Values with Tree Explainer
#model_tree = model1
model_tree = model2
explainer = shap.TreeExplainer(model_tree, X_train_transformed)
shap_values = explainer.shap_values(X_test_transformed)

In [ ]:
# Display SHAP summary plot
shap.summary_plot(shap_values, X_test_transformed, plot_type="bar")

In [ ]:
# Create local SHAP explanations
shap.initjs()

for i in range(0,12):
    print(i)
    shap_display = shap.force_plot(explainer.expected_value, shap_values[i, :], X_test_transformed.iloc[i, :])
    display(shap_display)

## LIME

In [ ]:
# Init LIME explainer
explainer = lime.lime_tabular.LimeTabularExplainer(X_test_transformed.to_numpy(),
                                                   feature_names= X_test_transformed.columns,
                                                   class_names=['BlackBox'],
                                                   verbose=True,
                                                   mode='regression')

In [ ]:
# Create local LIME explanations
#predictor = model1
predictor = model2

for i in range(0,12):
    print(i)
    explanation = explainer.explain_instance(X_test_transformed.iloc[i], predictor.predict, num_features=len(X_test_transformed.columns))
    explanation.show_in_notebook(show_table=True)